In [1]:

import sys  
sys.path.insert(0, r"C:\\Users\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\druida_V01\\src\\")

import os

from __future__ import print_function
#from Utilities.SaveAnimation import Video



from druida import Stack
from druida import setup

from druida.DataManager import datamanager
from druidaHFSS.modules import tools
from druida.tools import utils

import random
import numpy as np
import pandas as pd
from numpy.linalg import norm
from scipy.signal import find_peaks
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAccuracy




import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.offsetbox import OffsetImage,AnchoredOffsetbox
import matplotlib.image as image

from IPython.display import HTML

from glob import glob

import argparse



